# New case studies for Robot Dance paper

In [ ]:
import os
import datetime
from importlib import reload
import pandas as pd
import numpy as np
import numpy.linalg as la
import scipy.stats as stats
import matplotlib.pylab as plt
from matplotlib import rc
rc("text", usetex=True)
rc("font", family="serif")

import run_robot
import prepare_data
reload(run_robot)

## Define Subnotification factor

Between 21st and 29th of July the city of São Paulo made public the result of a research that [17.9% of its population](https://www1.folha.uol.com.br/equilibrioesaude/2020/08/em-sao-paulo-22-dos-moradores-dos-bairros-mais-pobres-ja-pegaram-coronavirus.shtml) had alredy had Covid-19. Here we use that number to find out a reasonable subnotification factor.

In [ ]:
basic_prm = prepare_data.save_basic_parameters(min_level=0.8, rep=2.5, ndays=30)
subnot_factor = 11.6
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["SP"], 10000000, subnot_factor, 1, "data/manuscript/manuscript_covid_with_drs_07_29.csv")
cities_data



Now, we define some important decisions:

* The basic reproduction rate (R0). The original literature and our own estimates suggest 2.5. But this value seems high nowdays when people are wearing masks, have learned stricter hygiene habits (more hand wahing), and do basic social distancing. I am trying now with 1.8.

* Horizon of simulation: we use a little more than one year because after that we should probably have a vacine and the game changes completely.

* Mean stay in ICU: mean time in ICU, it will also be used to select the right time series to estimate the number of ICU needed below. We are using 7.

* Lockdown level: what is the reproduction level achievable by a strict lockdown. We are using 0.8. Should be smaller than 1.

In [ ]:
# Define the basic data for the case studies

# Basic reproduction number
basic_rep = 1.8

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 14*2*14

# Mean time in ICU
time_icu = 7

# Lockdown level
lock_level = 0.8

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(min_level=lock_level, rep=basic_rep, time_icu=time_icu, ndays=ndays)

# Compute initial values

# For cities
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["Araçatuba", "São José Do Rio Preto"], 500000, 1)
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["São José Do Rio Preto"], 25000, 6, 1)

# For DRS
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", [], 000000, subnot_factor, 1, "data/manuscript/manuscript_covid_with_drs_07_01.csv")

# Sub-groups for figures
sp = ["SP"]
sp_so = sp + ["SW"]
masp_names = sp + ["E", "N", "W", "SE", "SW"]

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.ones((ncities, ndays))
cities_data

## Add information on the time series that estimate the need of ICUs

We are using the time series adjusted considering that the mean ICU stay is 7 days (which lead to larger ICU capacity).

In [ ]:
if basic_prm["time_icu"] == 11:
    # Time series adjusted considering the mean ICU time is 11 days
    ts_sp = np.array([0.0074335, 0.01523406, -0.00186355, 0.0, 1.67356018, -0.68192908, np.sqrt(0.00023883),
        0.007682840158843, 0.007536060983504])
    ts_notsp = np.array([0.00520255, 0.01532709, 0.00044498, 0.0, 1.75553282, -0.76360711, np.sqrt(3.567E-05),
        0.005426447471187, 0.005282217308748])
elif basic_prm["time_icu"] == 7:
    # Time series adjusted considering the mean ICU time is 7 days
    ts_sp = np.array([0.01099859, 0.02236023, 0.00370254, 0.0, 1.79119571, -0.80552926, np.sqrt(0.00034005),
        0.011644768910252, 0.011221496171591])
    ts_notsp = np.array([0.0076481, 0.0218084, 0.00367839, 0.0, 1.81361379, -0.82550856, np.sqrt(8.028E-05),
        0.007907216664912, 0.007721801045322])
else:
    raise NotImplementedError

# Index of the cities that form the Metropolitan area of São Paulo
MASP = np.array([7, 10, 15, 16, 17, 22]) - 1

ts_drs = np.ones((len(cities_data), len(ts_notsp)))
ts_drs *= ts_notsp
ts_drs[MASP, :] = ts_sp
ts_drs = pd.DataFrame(data=ts_drs, index=cities_data.index, columns=[
    "rho_min", "rho_max", "intercept", "trend", "phi_1", "phi_2", "sigma_omega", "state0", "state_less_1"
])
ts_drs["confidence"] = 0.9
ts_drs["time_icu"] = time_icu
cities_data = pd.concat([cities_data, ts_drs], axis=1)
cities_data





In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, pools=None, verbosity=1):
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif, pools, verbosity=verbosity)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, target, verbosity=verbosity)
    result = pd.read_csv(result_file, index_col=[0, 1])
    run_robot.plot_result(basic_prm, result, figure_file[:-4] + "_sp.png", hammer_data.loc[sp, "duration"].values, 
        cities_data["start_date"][0], sp)
    plt.savefig(figure_file[:-4] + "_sp.png", dpi=150, bbox_inches='tight')
    run_robot.plot_result(basic_prm, result, figure_file[:-4] + "_spso.png", hammer_data.loc[sp_so, "duration"].values, 
        cities_data["start_date"][0], sp_so)
    plt.savefig(figure_file[:-4] + "_sp_so.png", dpi=150, bbox_inches='tight')
    run_robot.plot_result(basic_prm, result, figure_file[:-4] + "_rmsp.png", 
        hammer_data.loc[masp_names, "duration"].values, cities_data["start_date"][0], masp_names)
    plt.savefig(figure_file[:-4] + "_rmsp.png", dpi=150, bbox_inches='tight')


## Case 1: 14 day window, no alternation, with mobility (5 neighbors)

In [ ]:
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/manuscript/manuscript_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M.loc["SP", "SW"], M.loc["SW", "SP"]

In [ ]:
%%time
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_withmobility.csv"
figure_file = "results/window_14_noalt_withmobility.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)

In [ ]:
def perturb(cities_data, low=0.9, up=1.1):
    cities_data = cities_data.copy()
    values = cities_data.iloc[:, :4].values
    values = values * np.random.uniform(low, up, size=values.shape)
    cities_data.iloc[:, :4] = (values.T / la.norm(values, 1, axis=1)).T
    return cities_data

original_data = cities_data.copy()


In [ ]:
cities_data = perturb(original_data)
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/manuscript/manuscript_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M.loc["SP", "SW"], M.loc["SW", "SP"]
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_withmobility2.csv"
figure_file = "results/window_14_noalt_withmobility2.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)

In [ ]:
cities_data = perturb(original_data)
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/manuscript/manuscript_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M.loc["SP", "SW"], M.loc["SW", "SP"]
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_withmobility3.csv"
figure_file = "results/window_14_noalt_withmobility3.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)

In [ ]:
cities_data = perturb(original_data)
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/manuscript/manuscript_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M.loc["SP", "SW"], M.loc["SW", "SP"]
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_withmobility4.csv"
figure_file = "results/window_14_noalt_withmobility4.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)

In [ ]:
cities_data = perturb(original_data)
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/manuscript/manuscript_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M.loc["SP", "SW"], M.loc["SW", "SP"]
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_withmobility5.csv"
figure_file = "results/window_14_noalt_withmobility5.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)